<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/GS_v0_2_con_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/base-diputados -f base_formateada_snowball.csv
!kaggle datasets download -d leonardocaravaggio/base-diputados -f vocabulary_snowball.voc

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
base_formateada_snowball.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
vocabulary_snowball.voc.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [5]:
import pandas as pd
base=pd.read_csv('base_formateada_snowball.csv')

<ipython-input-5-9ac41a69d66e>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('base_formateada_snowball.csv')


In [6]:
import pickle

with open('vocabulary_snowball.voc', 'rb') as f:
    inter = pickle.load(f)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2) ,vocabulary=inter)
matrix = vect.fit_transform(base['tokens'])

In [8]:
len(vect.vocabulary_)

357780

In [9]:
# Esta celda tarda unos 120 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=30, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=30,
                          random_state=0)

In [10]:
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

# Topics por weights

In [11]:
disc_weights=matrix*lda.components_.transpose()

In [12]:
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [13]:
base.fecha=pd.to_datetime(base.fecha, infer_datetime_format=True)

In [14]:
# Por LEGISLATURA por WEIGHTS
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()/base_legislatura['Topic_weight'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights.csv')

In [15]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()/base_year['Topic_weight'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_weights.csv')

# FREX

In [16]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [17]:
frex_df=pd.DataFrame(frex(lda))

In [18]:
# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['id']=range(0,334421)
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [19]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()/base_legislatura['Topic_FREX'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex.csv')

In [20]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()/base_year['Topic_FREX'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_frex.csv')

# Bigramas por weights

In [21]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [22]:
bigramas.to_csv('bigramas_por_weights_30topics.csv')

# Bigramas por frex

In [23]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [24]:
bigramas.to_csv('bigramas_por_frex_30topics.csv')

# Bigramas para 10 Topics

In [25]:
# Esta celda tarda unos 25 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, random_state=0)

In [26]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [27]:
bigramas.to_csv('bigramas_por_weights_10topics.csv')

In [28]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [29]:
bigramas.to_csv('bigramas_por_frex_10topics.csv')

# Bajada

In [30]:
from google.colab import files
files.download('topics_por_legislatura_por_weights.csv')
files.download('topics_por_año_por_weights.csv')
files.download('topics_por_legislatura_por_frex.csv')
files.download('topics_por_año_por_frex.csv')
files.download('bigramas_por_weights_30topics.csv')
files.download('bigramas_por_frex_30topics.csv')
files.download('bigramas_por_weights_10topics.csv')
files.download('bigramas_por_frex_10topics.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>